# 머신러닝 - 모델 평가 Part2

주제
- 데이터 분할
- K-Fold 교차 검증(Cross Validation)
- 하이퍼 파라미터 튜닝(Hyperparameter Tuning)
- 앙상블 학습(Ensemble Learning)
- 랜덤 포레스트(RandomForest)


# 예제 실습 - K-Fold 분할

In [2]:
from sklearn.model_selection import KFold
import numpy as np

data = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6])
kfold = KFold(n_splits=3,shuffle=True,random_state=42)

#kfoild
# 데이터 교차로 분할하기
for train,test in kfold.split(data):
    print(f"train: {data[train]} test: {data[test]}")


train: [0.3 0.4 0.5 0.6] test: [0.1 0.2]
train: [0.1 0.2 0.4 0.5] test: [0.3 0.6]
train: [0.1 0.2 0.3 0.6] test: [0.4 0.5]


# 예제 실습 - K-Fold 교차 검증

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

# 데이터셋 읽기
df = pd.read_csv("diabetes.csv")
print(df.shape)

# feature, target(Outcome) 분리
X = df.drop("Outcome",axis=1)
y = df["Outcome"] # 당뇨병 여부 (0: 당뇨병 X, 1: 당뇨병 O)

#1. Hold-out방법으로 KNN모델 평가해보기
# 훈련/테스트 데이터 분할(8:2), stratify 옵션 추가
X_train,X_test,y_train,y_test = train_test_split(
    X,y,test_size=0.2, random_state=42,stratify=y
)
#stratify=y : 원본 타겟 변수의 각 클래스 비율을 훈련/테스트에도 각각 유사한 비율로 분할하기

# K=3 기본 KNN 모델 생성 및 훈련 
knn = KNeighborsClassifier(n_neighbors=3) # K=3
knn.fit(X_train,y_train)

# 성능 평가
y_pred = knn.predict(X_test)
print(knn.score(X_test,y_test))
#or accuracy_score(y_test,y_pred)
# 교차 검증
cv_scores = cross_val_score(knn,X,y,cv=5) 
##1.모델 2.x전체데이터 3. y 전체 데이터, cv =K Fold의  K값(분할 개수)
#Scoring: 평기 기준 지표; (분류의 디폴트: accuracy)
#cv_scores: 각 폴드에 대한 평가 점수들에 대한 결과 리스트
print(cv_scores)
print(np.mean(cv_scores))

(768, 9)
0.6948051948051948
[0.68181818 0.69480519 0.75324675 0.75163399 0.68627451]
0.7135557253204311


# 예제 실습 - 하이퍼 파라미터 튜닝

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
import pandas as pd

# 데이터셋 읽기
df = pd.read_csv('diabetes.csv')
print(df.shape)

# feature, target 분리
X = df.drop('Outcome', axis=1)
y = df['Outcome'] # 당뇨병 여부 (0: 당뇨병 X, 1: 당뇨병 O)

# 훈련/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
#여기서 Train : 하이퍼 파라미터 찾기 및  모델 학습을 위함
# 최종적으로 최적의 하이퍼 파라미터로  학습된 모델을 평가하기 위한 테스트 데이터

knn = KNeighborsClassifier() #Base Model
param_grid = {"n_neighbors":np.arange(1,25)} # 1~24 k를 탐색
# 각 하이퍼 파라미터 마다의 시험하고자 하는 범위 (key:value = hyper:range)

knn_gscv = GridSearchCV(knn,param_grid,cv =5)
# 5-Fold로 교차검증하면서 튜닝
knn_gscv.fit(X_train,y_train)
print(knn_gscv.best_params_) 
# best_params_: 지정한 parm_grid에 대해 각 최적의  하이퍼 파라미터 값 변환
print(knn_gscv.best_score_)
#best_score_:Train 데이터 안에서 수행된 CV(교차검증) 평균 점수(Accuracy)

best_model = knn_gscv.best_estimator_
#best_estimator_ : 최적의 파라미터로 학습이 된 모델

#최적의 하이퍼 파라미터를 갖는 모델로 성능 평가
y_pred = best_model.predict(X_test)
print(accuracy_score(y_test,y_pred))


# Base 모델 생성

# 튜닝할 파라미터 범위 설정

# 5-Fold CV와 GridSearch로 튜닝

# 구해진 최적의 모델

# 해당 모델로 예측 및 평가 진행


(768, 9)
{'n_neighbors': np.int64(12)}
0.7736771957883513
0.7012987012987013


# 예제 실습 - 성능 향상

In [3]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

df = pd.read_csv("diabetes.csv")

X = df.drop("Outcome", axis=1)
y = df["Outcome"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

#Train : 하이퍼 파라미터 튜닝과 최적의 모델 학습을 위함
#Test: 최적의 모델을 평가하기 위함


# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)# 훈련 데이터 스케일링 
X_test_scaled = scaler.transform(X_test) # 테스트  데이터 스케일링


# 기본적인 모델 학습 
model = KNeighborsClassifier()
model.fit(X_train_scaled,y_train)


y_pred = model.predict(X_test_scaled)
print(f"기본 knn정확도 : {accuracy_score(y_test,y_pred)}")

# 하이퍼 파라미터 조합
param_grid = {
    'n_neighbors': list(range(3, 31, 2)), # 홀수 k 
    'weights': ['uniform', 'distance'],   #uniforme(디폴트), distance : 가까운 이웃에 가중치
    'metric': ['minkowski'],
    'p': [1, 2]  # manhattan, euclidean
}


# Base Model 생성
knn = KNeighborsClassifier()

# 튜닝 
grid = GridSearchCV(
    estimator=knn,  # 어떤 모델에 대해서 수행할 것인지 모델 객체
    param_grid=param_grid, # 하이퍼파라미터 격자 조합
    cv=5,             # 5-Fold 교차 검증
    scoring="accuracy"  # 평가 및 최적의 파라미터를 찾을 때의 기준 지표
)
grid.fit(X_train_scaled, y_train)

print(f"최적의 하이퍼 파라미터 :{grid.best_params_}")
print(f"교차 검증 최고 정확도 :{grid.best_score_}")
# 얻은 최적의 모델
best_model = grid.best_estimator_
# 최적의 모델로 예측 및 평가 진행

pred_best = best_model.predict(X_test_scaled)
print(f"최적 모델 정확도{accuracy_score(y_test,pred_best)}")




기본 knn정확도 : 0.7012987012987013
최적의 하이퍼 파라미터 :{'metric': 'minkowski', 'n_neighbors': 15, 'p': 1, 'weights': 'uniform'}
교차 검증 최고 정확도 :0.7687724910035986
최적 모델 정확도0.7727272727272727


# 예제 실습 - 앙상블; 랜덤포레스트

Iris 데이터셋으로 품종(Species)을 예측하는 RandomForestClassifier

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

# 1. 데이터 불러오기 및 feature/target 분리
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

# 2. 훈련/테스트 데이터 분할(8:2, stratify)
None

# 3. 랜덤포레스트 모델 생성 및 학습
# - n_estimators=200
# - max_depth=None
# - random_state=42
model = None
None

# 4. 예측 및 평가
pred = None

print("Accuracy:", accuracy_score(None, None))
print("Classification Report:", classification_report(None, None))

# 5. Feature Importance 시각화
importances = model.None # 중요도 가져오기
features = X.columns

# 컬럼별 중요도 점수 막대 그래프 시각화
plt.bar(None, None)
plt.xticks(rotation=45)
plt.title("Feature Importances (RandomForest Classifier)")
plt.show()

# 6. GridSearchCV를 통한 하이퍼파라미터 튜닝
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [None, 5, 10],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2]
}

# GridSearch로 하이퍼 파라미터 튜닝하기
# - CV: 5
# - 성능 평가 지표 기준: 정확도
grid = GridSearchCV(
    RandomForestClassifier(random_state=42),
    None,
    cv=None,
    scoring=None,
)

# 튜닝 수행하기
grid.fit(None, None)

# 최적의 파라미터들과 점수 출력
print("Best Params:", grid.None)
print("Best CV Score:", grid.None)

# 7. 최적 모델로 다시 평가
best_model = grid.None
best_pred = best_model.None

print("[Best Model Accuracy]:", accuracy_score(None, None))

## 타이타닉 생존자 예측

요구사항

1. titanic3.csv 데이터셋을 읽어옵니다.
2. 결측치는 간단히 처리합니다.
  - age : 평균 또는 중앙값으로 대체
  - embarked : 최빈값으로 대체
3. 사용할 feature는 다음과 같이 한다.
  - pclass, sex, age, sibsp, parch, fare, embarked
4. 범주형 변수(sex, embarked)는 OneHotEncoder로 처리한다.
5. survived를 타깃으로,
  - train_test_split(..., stratify=y, test_size=0.2)로 분할한다.
6. RandomForestClassifier에 대해 아래 파라미터로 GridSearchCV를 수행한다.
  - n_estimators: `[100, 200]`
  - max_depth: `[None, 5, 10]`
  - min_samples_split: `[2, 5]`
  - cv=5, scoring='accuracy'
7. 최적 모델에 대해 
  - 교차 검증 평균 정확도, 
  - 테스트 세트 정확도를 출력한다.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 1. 데이터 불러오기
df = pd.read_csv("titanic3.csv")

# 2. 결측치 처리
df["age"] = None
df["embarked"] = None

# 3. feature와 target 분리
features = ["pclass", "sex", "age", "sibsp", "parch", "fare", "embarked"]
X = None
y = None

# 4. Stratified train/test 분할 (8:2)
None

# 5. 범주형 인코딩: get_dummies 사용
#    - train에는 fit 개념으로 get_dummies
#    - test는 같은 컬럼 구조로 맞춰줘야 함
categorical_cols = ["sex", "embarked"]

# 5-1. 훈련 데이터에 대한 인코딩 
X_train_encoded = pd.get_dummies(
    None,
    columns=None,
    drop_first=True  # 더미 변수 함정 방지를 위해 첫 카테고리 드롭
)
# 5-2. 테스트 데이터에 대한 인코딩
X_test_encoded = pd.get_dummies(
    None,
    columns=None,
    drop_first=True
)

# train과 test의 컬럼(더미 변수)이 다를 수 있으므로, test를 train에 맞춰 재정렬
X_test_encoded = X_test_encoded.reindex(
    columns=X_train_encoded.columns,
    fill_value=0
)


# 6. RF + GridSearchCV (전처리 분리, 모델만 튜닝)
rf = RandomForestClassifier(random_state=42)

# 6-1. 탐색할 최적의 하이퍼 파라미터 격자 정의
param_grid = None

# GridSearch 정의
grid_search = GridSearchCV(
    rf,
    param_grid=None,
    cv=None,
    scoring=None,
    n_jobs=-1
)

# 튜닝 수행하기
grid_search.fit(None, None)

print(f"최적 하이퍼파라미터: {grid_search.None}")
print(f"최적 CV 평균 정확도: {grid_search.None:.4f}")

# 최적의 튜닝된 모델 가져오기
best_model = None

# 7. 테스트 세트 평가
y_pred = best_model.None

test_acc = accuracy_score(None, None)
print(f"테스트 정확도: {test_acc:.4f}")


## 은행 정기예금 가입 예측

은행에서는 고객의 신규 정기예금 가입 여부(y)를 예측하고자 한다.
예측 대상은 0 = 가입 X / 1 = 가입 O 이다.

| 컬럼       | 설명          |
| -------- | ----------- |
| age      | 나이          |
| job      | 직업          |
| marital  | 결혼 여부       |
| balance  | 잔고          |
| duration | 통화 시간       |
| campaign | 캠페인 연락 횟수   |
| deposit  | 목표 변수 (0/1) |

1. 데이터 불러오기 및 기초 확인
  - bank.csv 읽기
2. 범주형 인코딩
  - job, marital → pd.get_dummies(drop_first=True)
  - 이진 데이터는 그대로 사용
3. 스케일링
  - StandardScaler:
  - age, balance, duration, campaign 적용
4. 훈련/테스트 데이터 분할
  - stratify=y
  - test_size=0.2
5. 기본 모델: Logistic Regression
  - 학습
  - Accuracy, F1-score 출력
6. 교차검증
  - 5-Fold
  - scoring = "accuracy"
7. GridSearchCV (KNN 대상)
  - 튜닝할 파라미터:
    ```
    
        "n_neighbors": [3,5,7,9,11],
        "weights": ["uniform","distance"],
        "metric": ["euclidean","manhattan"]
    
    ```
  - 5-Fold CV
  - scoring="accuracy"
  - 최적 파라미터 & 성능 출력
8. Confusion Matrix + ROC Curve 시각화
  - heatmap
  - ROC curve & AUC 출력

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix, roc_auc_score, roc_curve
from sklearn.neighbors import KNeighborsClassifier

# 1. 데이터 셋을 읽고, 데이터 구조 파악
df = None
df = df[['age', 'job', 'marital', 'balance', 'duration', 'campaign', 'deposit']] # 일부 feature들만 활용

print(df.head(), df.info(), df.isnull().sum())

# 2. 인코딩
df = pd.None(df, columns=["job","marital"], drop_first=True)

# 3. 수치형 컬럼에 대한 스케일링
scale_cols = ["age","balance","duration","campaign"]

scaler = None
df[None] = scaler.None(df[None])

# 4. 분할 (target: deposit)
X = None
y = df[None].map({'no' : 0, 'yes' : 1})

# 4-1. 훈련/테스트 분할 (8:2), stratify
None
# 5. Logistic Regression 학습 및 성능 평가 진행
lr = None(max_iter=2000)
lr.fit(None,None)
pred_lr = lr.predict(None)

print("LR Accuracy:",accuracy_score(None,None))
print("LR F1:",f1_score(None,None))

# 6. 교차검증 - CV
# - cv: 5
# - 성능 평가 지표 기준: 정확도
cv_scores = None
print("CV scores:",None)
print("CV Mean:",None)

# 7. KNN과 GridSearchCV으로 하이퍼 파라미터 튜닝 수행
param_grid = None

# Base Model
knn = KNeighborsClassifier()
# GridSearch 생성 및 튜닝 수행
# - cv: 5
# - 성능 평가 지표 기준: 정확도
grid = None
None

print("Best Params:",grid.None)
print("Best CV Score:",grid.None)

# 8. Confusion Matrix
# 8-1. 최적의 모델 가져오고 예측 수행
best_knn = None
pred_knn = None
# 8-2. 혼동 행렬 생성
cm = None

# 8-3. 혼동 행렬 히트맵으로 시각화
None
plt.show()

# 9. ROC Curve
# 9-1. 최적의 모델로 예측 확률 중에서 양성(1)일 확률만 추출
proba = None
# 9-2. roc_curve 생성하기
fpr, tpr, thresholds = None

# # 9-3. ROC-Curve 시각화하기
plt.plot(None,None)
plt.plot([0,1],[0,1],'--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("KNN ROC Curve")
plt.show()

# 9-4. ROC-AUC 점수 출력
print("KNN AUC:", None)